In [11]:
from sklearn import metrics
!pip install scipy numpy matplotlib pandas sklearn tabulate seaborn folium geopy geopandas requests> /dev/null
%matplotlib notebook

In [12]:
# import importlib
# import sys
# importlib.reload(sys.modules['statistics.company'])
# Load libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib import pyplot
from sklearn.model_selection import train_test_split, learning_curve, permutation_test_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import normalize
from sklearn.metrics import classification_report, roc_auc_score, roc_curve, plot_roc_curve, auc
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import calinski_harabasz_score, davies_bouldin_score, silhouette_score
from sklearn.cluster import KMeans
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from tabulate import tabulate
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
import os
import sys
sys.path.insert(0, os.path.abspath('/home/jovyan/host-note/module-dir'))

In [13]:
import importlib
import sys
importlib.reload(sys.modules['dataset_provider.provider'])
from dataset_provider.provider import get_raw_dataset
from dataset_provider.config import create_global_config
df = get_raw_dataset(create_global_config())

GetRawDataset - Started
GetRawDataset - read_json - DONE
GetRawDataset - map_columns_to_csv - DONE
GetRawDataset - read_csv - DONE
Found 304 job ads without salary range or currency
(1249, 527)
        salary_from     salary_to
count   1249.000000   1249.000000
mean   11148.760608  15884.851081
std     4540.909690   5114.312359
min     1250.000000   1750.000000
25%     8000.000000  12100.000000
50%    10000.000000  16000.000000
75%    14000.000000  19000.000000
max    44000.000000  50000.000000
Found 8 countries and 4 currencies!
Dropping foreign countries and translating currencies...


/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3254: DtypeWarning: Columns (24,25,27,28,29,30,32,35,36,37,38,40,43,45,46,48,49,50,53,58,60,61,62,63,64,66,68,69,70,71,73,74,76,77,78,80,82,83,84,86,88,90,91,92,93,94,96,97,98,102,103,105,107,108,109,110,111,112,113,114,115,119,120,121,123,124,126,127,130,131,132,133,134,135,138,141,142,143,144,145,148,149,151,152,153,154,155,156,161,162,163,165,166,167,170,171,172,173,177,180,182,185,186,187,188,189,190,191,194,195,196,197,198,200,201,206,207,208,209,210,211,212,213,214,215,218,220,221,222,223,224,225,226,227,229,231,232,235,236,238,239,240,241,243,244,245,249,250,251,254,257,258,259,264,265,269,270,272,273,275,276,279,282,283,287,288,289,291,292,294,295,297,298,299,300,301,305,308,312,313,315,317,318,319,321,322,323,326,327,332,333,334,337,338,339,340,341,343,344,345,348,351,352,353,355,356,357,358,359,360,362,363,365,366,368,369,370,371,372,373,374,375,378,379,380,385,387,391,393,396,397,398,399,400,401,402,404,

Unique countries: 1, currencies: 1, observations: 1230


---------------------------------------------------------------------------

# Regression

In [14]:
print(df.columns)
def move_col_to_end(df, col):
    cols_at_end = [col]
    return df[[c for c in df if c not in cols_at_end] 
            + [c for c in cols_at_end if c in df]]

Index(['Unnamed: 0', 'title', 'city', 'country_code', 'marker_icon', 'remote',
       'company_name', 'company_size', 'experience_level', 'salary_from',
       ...
       'narzędzi do prototypowania', 'nestJS', 'Oracle DB', 'ownership',
       'people-focused design mindset', 'self-organisation', 'Servers', 'soar',
       'Vert.x', 'workshop'],
      dtype='object', length=527)


In [115]:
cl_df = df.copy()
for col in ["employment_type", "remote", "salary_to", "salary_from", "Unnamed: 0", "title", "company_size", "country_code", "marker_icon", "company_name",
            "latitude", "longitude", "salary_currency", "published_at", "remote_interview", "id", "skills"]:
                cl_df=cl_df.drop(col, axis=1)
                
cl_df.columns
cl_df = cl_df.astype(str)
ass_df = [[x for x in row if x != "0"] for row in cl_df.values]
ass_df[0]

['Szczecin', 'senior', 'JavaScript', 'Node.js', 'TypeScript']

In [116]:
!pip install mlxtend > /dev/null

In [121]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

def filter_by_skill(data_frame, skill):
    cp = data_frame.copy()
    return cp.loc[(cp['antecedents'].apply(lambda x: x.issuperset({skill}))) | (cp['consequents'].apply(lambda x: x.issuperset({skill})))]

def require_antecetends(data_frame, skill):
    cp = data_frame.copy()
    return cp.loc[(cp['antecedents'].apply(lambda x: x.issuperset({skill})))]

# filter_type = 'all' or 'antecedents'
def assocation_rules(min_conf, min_supp, skill_filter=None, filter_type='all'):
    te = TransactionEncoder()
    te_ary = te.fit(ass_df).transform(ass_df)
    f_df = pd.DataFrame(te_ary, columns=te.columns_)

    frequent_itemsets = apriori(f_df, min_support=min_supp, use_colnames=True)
    rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=min_conf)
    if skill_filter:
        if filter_type == 'all':
            return filter_by_skill(rules, skill_filter).sort_values(['confidence', 'support'], ascending=False)
        else:
            return require_antecetends(rules, skill_filter).sort_values(['confidence', 'support'], ascending=False)
    else:
        return rules.sort_values(['confidence', 'support'], ascending=False)
        
assocation_rules(0.4, 0.008, 'senior')

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
164,"(Hibernate, senior)",(Java),0.008130,0.175610,0.008130,1.000000,5.694444,0.006702,inf
255,"(Warszawa, Spring, senior)",(Java),0.013008,0.175610,0.012195,0.937500,5.338542,0.009911,13.190244
181,"(Spring, senior)",(Java),0.030894,0.175610,0.028455,0.921053,5.244883,0.023030,10.442276
109,"(Java, AWS)",(senior),0.008943,0.364228,0.008130,0.909091,2.495942,0.004873,6.993496
142,"(Django, senior)",(Python),0.013008,0.073171,0.010569,0.812500,11.104167,0.009617,4.943089
206,"(TypeScript, senior)",(JavaScript),0.017073,0.190244,0.012195,0.714286,3.754579,0.008947,2.834146
3,(AWS),(senior),0.035772,0.364228,0.024390,0.681818,1.871956,0.011361,1.998142
173,"(Java, Kraków)",(senior),0.036585,0.364228,0.024390,0.666667,1.830357,0.011065,1.907317
59,(Microservices),(senior),0.012195,0.364228,0.008130,0.666667,1.830357,0.003688,1.907317
113,"(Android, senior)",(Kotlin),0.012195,0.023577,0.008130,0.666667,28.275862,0.007843,2.929268
